In [1]:
# Cell 2: Import required libraries
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras import layers, models

In [2]:
# Cell 3: Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# For binary classification: 1 = automobile (recyclable), 0 = cat (non-recyclable)
class_map = {1: 'automobile', 3: 'cat'}
train_filter = np.isin(y_train, list(class_map.keys()))
test_filter = np.isin(y_test, list(class_map.keys()))

x_train_bin = x_train[train_filter.squeeze()]
y_train_bin = y_train[train_filter].squeeze()
x_test_bin = x_test[test_filter.squeeze()]
y_test_bin = y_test[test_filter].squeeze()

# Map labels to 0 and 1
y_train_bin = (y_train_bin == 1).astype(np.float32)
y_test_bin = (y_test_bin == 1).astype(np.float32)

# Normalize images
x_train_bin = x_train_bin.astype('float32') / 255.0
x_test_bin = x_test_bin.astype('float32') / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
# Cell 4: Define a simple CNN model
model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    x_train_bin, y_train_bin,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - accuracy: 0.7957 - loss: 0.4604 - val_accuracy: 0.9110 - val_loss: 0.2238
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9271 - loss: 0.1919 - val_accuracy: 0.9050 - val_loss: 0.2350
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.9341 - loss: 0.1723 - val_accuracy: 0.9320 - val_loss: 0.1769
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 46ms/step - accuracy: 0.9435 - loss: 0.1398 - val_accuracy: 0.9280 - val_loss: 0.1786
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9413 - loss: 0.1510 - val_accuracy: 0.9420 - val_loss: 0.1505


In [4]:
# Cell 5: Evaluate on test set
test_loss, test_acc = model.evaluate(x_test_bin, y_test_bin, verbose=2)
print(f'Test accuracy: {test_acc:.4f}')

63/63 - 0s - 7ms/step - accuracy: 0.9500 - loss: 0.1423
Test accuracy: 0.9500


In [5]:
# Cell 6: Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('edge_model.tflite', 'wb') as f:
    f.write(tflite_model)
print('TFLite model saved as edge_model.tflite')

Saved artifact at '/tmp/tmpovl5lquq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  133148991429072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991430224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991428496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991427344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991429648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991429264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991430608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133148991429840: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved as edge_model.tflite


In [6]:
# Cell 7: Run inference with TFLite model
interpreter = tf.lite.Interpreter(model_path='edge_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on 100 samples
num_samples = 100
total_time = 0
correct = 0

for i in range(num_samples):
    input_data = np.expand_dims(x_test_bin[i], axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    start = time.time()
    interpreter.invoke()
    end = time.time()
    output = interpreter.get_tensor(output_details[0]['index'])
    pred = (output[0][0] > 0.5).astype(np.float32)
    if pred == y_test_bin[i]:
        correct += 1
    total_time += (end - start)

print(f'TFLite accuracy on {num_samples} samples: {correct/num_samples:.4f}')
print(f'Average inference time per image: {total_time/num_samples*1000:.2f} ms')

TFLite accuracy on 100 samples: 0.9600
Average inference time per image: 0.11 ms
